In [131]:
import numpy as np
import seaborn as sns
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [132]:
df = pd.read_csv('USA_unemployment_Data.csv')
df.head()

,Date,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,...,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming,National
0,Jan-76,7.5,9.3,11.5,8.8,10.4,7.1,10.9,8.7,9.2,...,6.8,6.3,6.8,10.0,7.1,10.4,9.1,6.8,5.9,8.8
1,Feb-76,7.7,9.7,10.6,8.6,10.1,6.6,10.8,8.6,9.3,...,6.7,6.2,7.3,9.7,7.0,10.2,9.0,7.6,5.0,8.7
2,Mar-76,6.3,8.7,9.8,7.5,9.4,6.9,9.9,8.3,8.6,...,6.1,5.9,6.4,9.4,6.4,9.5,8.3,6.5,4.7,8.1
3,Apr-76,5.7,7.5,9.2,6.7,8.8,5.2,9.4,7.9,7.7,...,5.5,5.2,5.5,9.0,5.5,8.4,7.6,5.5,4.1,7.4
4,May-76,5.2,6.5,8.4,6.2,7.9,4.5,8.8,7.5,7.6,...,5.0,5.1,4.7,8.0,5.3,7.5,6.8,4.8,3.4,6.8


In [133]:
train_dates = pd.to_datetime(df['Date'], format="%b-%y", errors="coerce").fillna(pd.to_datetime(df['Date'], format="%y-%b", errors="coerce"))

In [149]:
ar=np.array(df_for_training)
ar[:,0][0:500]

array([ 7.5,  7.7,  6.3,  5.7,  5.2,  7.4,  7.2,  7.1,  6.7,  6.6,  6.7,
        6.6,  7.7,  8.4,  7.4,  6.7,  6.5,  8.4,  7.7,  7.6,  6.7,  6.4,
        6.4,  6. ,  7. ,  7. ,  6.2,  5.5,  5.3,  7. ,  6.7,  6.7,  6.5,
        6.2,  6.5,  6.7,  7. ,  7.6,  6.9,  6.5,  6.3,  8.1,  7.4,  7.9,
        7.2,  7.3,  7.2,  7.1,  8.1,  8.2,  7.6,  7.7,  8.1,  9.6,  9.7,
        9.8,  9.1,  9.3,  9.3,  9.1, 10.3, 10.6,  9.7,  9.3,  9.5, 11. ,
       10.6, 10.9, 11. , 11.5, 11.8, 12.3, 13.3, 13.7, 13.1, 12.9, 12.9,
       14.4, 14.4, 14.5, 14.2, 14.5, 14.8, 14.8, 15.6, 15.6, 14.5, 13.8,
       13.5, 14.7, 13.7, 13.9, 13.3, 12.8, 12.3, 12.1, 12.8, 12.8, 11.8,
       11.3, 10.6, 11.6, 11.6, 11.5, 10.8, 10.5, 10. , 10.1, 10.9, 10.6,
        9.9,  9.2,  9. , 10.1,  9.6,  9.4,  9.1,  9.1,  8.9,  9. ,  9.6,
       10.3,  9.8,  9.3,  9.2, 10.3,  9.6,  9.6,  9.5,  9.4,  9.3,  8.8,
        9.8,  9.7,  8.8,  7.9,  7.8,  8.4,  8. ,  8. ,  7.6,  7.6,  7.4,
        7.2,  7.9,  8. ,  7.2,  6.7,  6.8,  7.6,  7

In [143]:
cols = list(df)[1:]
df_for_training = df[cols].astype(float)
scaler = StandardScaler()
scaler = scaler.fit(df_for_training)
df_for_training_scaled = scaler.transform(df_for_training[:,0][0:540])

TypeError: '(slice(None, None, None), 0)' is an invalid key

In [135]:
trainX = []
trainY = []
n_future = 1
n_past = 3

for i in range (n_past, len(df_for_training_scaled)- n_future +1):
    trainX.append(df_for_training_scaled[i - n_past:i, 0:df_for_training.shape[1]])
    trainY.append(df_for_training_scaled[i + n_future - 1:i + n_future, 0])

trainX, trainY = np.array(trainX), np.array(trainY)
print('trainX shape == {}.'.format(trainX.shape))
print('trainY shape == {}.'.format(trainY.shape))

trainX shape == (552, 3, 52).
trainY shape == (552, 1).


In [136]:
model = Sequential()
model.add(LSTM(64, activation = 'relu', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
model.add(LSTM(32, activation = 'relu', return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(trainY.shape[1]))

model.compile(optimizer = 'adam', loss = 'mse')
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 3, 64)             29952     
                                                                 
 lstm_13 (LSTM)              (None, 3, 32)             12416     
                                                                 
 dropout_5 (Dropout)         (None, 3, 32)             0         
                                                                 
 dense_5 (Dense)             (None, 3, 1)              33        
                                                                 
Total params: 42,401
Trainable params: 42,401
Non-trainable params: 0
_________________________________________________________________


In [137]:
history = model.fit(trainX, trainY, epochs=10, batch_size=16, validation_split=0.1, verbose=1)

Epoch 1/10
31/31 [==============================] - 2s 14ms/step - loss: 0.5398 - val_loss: 0.9114
Epoch 2/10
31/31 [==============================] - 0s 4ms/step - loss: 0.2870 - val_loss: 0.9054
Epoch 3/10
31/31 [==============================] - 0s 4ms/step - loss: 0.2111 - val_loss: 0.9907
Epoch 4/10
31/31 [==============================] - 0s 4ms/step - loss: 0.1952 - val_loss: 0.9412
Epoch 5/10
31/31 [==============================] - 0s 4ms/step - loss: 0.1485 - val_loss: 0.8023
Epoch 6/10
31/31 [==============================] - 0s 4ms/step - loss: 0.1332 - val_loss: 0.8105
Epoch 7/10
31/31 [==============================] - 0s 4ms/step - loss: 0.1116 - val_loss: 0.7595
Epoch 8/10
31/31 [==============================] - 0s 4ms/step - loss: 0.1009 - val_loss: 0.8949
Epoch 9/10
31/31 [==============================] - 0s 4ms/step - loss: 0.1000 - val_loss: 0.7065
Epoch 10/10
31/31 [==============================] - 0s 4ms/step - loss: 0.0902 - val_loss: 0.7605


In [138]:
n_future=12
forcast_period_dates = pd.date_range(list(train_dates)[-1], periods=n_future,freq='1m' ).tolist()
forcast = model.predict(trainX[-n_future:])

1/1 [==============================] - 0s 264ms/step


In [139]:
forcast_copies = np.repeat(forcast, df_for_training.shape[1], axis=-1)
y_pred_future = scaler.inverse_transform(forcast_copies)[:,0]

forcast_dates = []
for time_i in forcast_period_dates:
    forcast_dates.append(time_i.date())

In [140]:
print(np.array(forcast_dates))

[datetime.date(2022, 3, 31) datetime.date(2022, 4, 30)
 datetime.date(2022, 5, 31) datetime.date(2022, 6, 30)
 datetime.date(2022, 7, 31) datetime.date(2022, 8, 31)
 datetime.date(2022, 9, 30) datetime.date(2022, 10, 31)
 datetime.date(2022, 11, 30) datetime.date(2022, 12, 31)
 datetime.date(2023, 1, 31) datetime.date(2023, 2, 28)]


In [141]:
df_forcast = pd.DataFrame({'Date':np.array(forcast_dates), 'Alabama':y_pred_future[:,0]})
df_forcast['Date']=pd.to_datetime(df_forcast['Date'])

original = df[['Date', 'Alabama']]
original['Date'] = pd.to_datetime(df['Date'], format="%b-%y", errors="coerce").fillna(pd.to_datetime(df['Date'], format="%y-%b", errors="coerce"))
# sns.lineplot(df_forcast['Date'], original['Alabama'])
# sns.lineplot(df_forcast['Date'], df_forcast['Alabama'])


F:\UNIMELB\Elements_of_data_processing\Anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [142]:
df_forcast

,Date,Alabama
0,2022-03-31,6.812989
1,2022-04-30,6.687788
2,2022-05-31,6.529402
3,2022-06-30,6.322900
4,2022-07-31,6.209394
5,2022-08-31,6.546832
6,2022-09-30,6.122738
7,2022-10-31,5.773018
8,2022-11-30,5.188392
9,2022-12-31,4.747318
